# Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import date

# Data Import & join
### Datasets: PALMS
First ensure that none of the files are duplicated by checking control sums.

In [2]:
start_year = 2016
start_month = 3
n_files = 59

df_temp = pd.read_csv("data/palms_report_data_2016_03.csv", index_col=0, encoding="ISO-8859-1")
sum_previous = df_temp.sum()

for i in range(n_files):
    month = (start_month + i) % 12 + 1
    month = str(month)
    if len(month) == 1:
        month = "0" + month
    else:
        pass
    year = start_year + ((start_month + i) // 12)
    
    df_temp = pd.read_csv(f"data/palms_report_data_{year}_{month}.csv", index_col=0, encoding="ISO-8859-1")
    sum_current = df_temp.sum()
    
    # If all of the column sums are the same, then sum of the boolean comparison on the left
    # will be equal to the number of columns seen on the right
    if (sum_previous == sum_current).sum() == sum_current.shape[0]:
        print("Duplicated readings")
        print(f"Current file: region-palms-report_{year}_{month}.csv")
        break
    else:
        sum_previous = sum_current

The loop hasn't been broken for any of the instances which means that the files are not duplicate. At least they are not positioned month by month but it is even more unlikely that a duplicated file has been saved in a file where the month differs by more than one.

Just to double-check lets check the condition for the same file.

In [3]:
(sum_current == sum_current).sum() == sum_current.shape[0]

True

As expected - everything is working correctly. Lets import the files and concatenate them.

In [4]:
start_year = 2016
start_month = 2
n_files = 60

df_palms = pd.DataFrame()

for i in range(n_files):
    month = (start_month + i) % 12 + 1
    month = str(month)
    if len(month) == 1:
        month = "0" + month
    else:
        pass
    year = start_year + ((start_month + i) // 12)
#     print(f"region-palms-report_{year}_{month}.csv")
    
    df_temp = pd.read_csv(f"data/palms_report_data_{year}_{month}.csv", index_col=0, encoding="ISO-8859-1")
    df_temp["palms_date"] = date(year, int(month), 1)

    df_palms = pd.concat([df_palms, df_temp])

column_list = df_palms.columns.tolist()
column_list = column_list[-3:-1] + column_list[:-3] + [column_list[-1]]
df_palms = df_palms[column_list]

df_palms.reset_index(inplace=True, drop=True)
df_palms

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,RRO,V,1-2-1,TYFCB,CEU,palms_date
0,733,1,3,0,0,0,0,0,0,0,0,0,1,0,0,2016-03-01
1,1150,1,3,0,0,0,0,0,0,0,0,0,0,0,0,2016-03-01
2,414,1,3,0,0,0,0,0,0,0,0,0,2,0,0,2016-03-01
3,1721,1,3,0,0,0,0,0,0,0,0,0,0,0,0,2016-03-01
4,1919,1,4,0,0,0,0,0,0,0,0,0,1,0,0,2016-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29211,2875,31,4,0,0,0,0,5,4,11,7,1,1,3913,2,2021-02-01
29212,276,31,4,0,0,0,0,3,2,1,0,0,3,4214,8,2021-02-01
29213,585,31,3,1,0,0,0,8,1,2,6,0,7,73,0,2021-02-01
29214,1223,31,3,1,0,0,0,2,4,7,1,0,10,1755,4,2021-02-01


### Dataset: database

In [5]:
df_database = pd.read_csv("data/database_data.csv", index_col=0, encoding="ISO-8859-1")
column_list = df_database.columns.tolist()

column_list = column_list[-3:-1] + column_list[:-3] + [column_list[-1]]
df_database = df_database[column_list]
df_database.head(10)

,user_ID,chapter_ID,Profession,Position,Join Date,Renewal Date,sponsor_ID
0,982,22,"Medical, Chiropractor",NaN,01/07/2004,01/12/2010,NaN
1,860,22,"Trades, Renovations-Remodeling",NaN,01/02/2008,01/12/2008,NaN
2,1970,22,"Food/Beverages, Chef",NaN,01/09/2010,01/12/2011,NaN
3,898,22,"Gifts, Gift Baskets",NaN,01/03/2007,07/04/2008,898.0
4,2204,22,"Mortgage, Mortgage Broker",NaN,01/07/2007,01/07/2008,NaN
5,1465,22,"Real estate services, Residential Real Estate ...",NaN,01/07/2007,01/04/2021,NaN
6,2377,22,"Insurance, Life,Health and Disability Insurance",NaN,01/07/2007,01/11/2009,NaN
7,2214,22,"Financial, Investment Advisor",NaN,01/07/2007,01/04/2009,NaN
8,832,22,"Health & Wellness, Massage Therapist",NaN,01/07/2007,01/07/2008,NaN
9,715,22,"Insurance, General-Motor Insurance",NaN,01/10/2007,01/10/2008,898.0


In [6]:
df_database.rename(columns={"Profession": "profession",
                            "Position": "position",
                            "Join Date": "join_date",
                            "Renewal Date": "renewal_date",}, 
                   inplace=True)


df_database["join_date"] = pd.to_datetime(df_database["join_date"], format='%d/%m/%Y', errors='coerce')
df_database["renewal_date"] = pd.to_datetime(df_database["renewal_date"], format='%d/%m/%Y', errors='coerce')
# df_database["sponsor_ID"] = pd.to_numeric(df_database["sponsor_ID"], errors='coerce', downcast='Int32')
df_database.tail()

,user_ID,chapter_ID,profession,position,join_date,renewal_date,sponsor_ID
3389,2265,37,"Construction, Builder/General Contractor",NaN,2021-03-01,2022-03-01,2512.0
3390,2736,35,"Legal & Accounting, Real Estate Law",NaN,2021-02-01,2022-02-01,NaN
3391,1827,35,"Construction, Electrician",NaN,2021-02-01,2022-02-01,2419.0
3392,257,35,"Finance & Insurance, Life,Health and Disabilit...",NaN,2021-03-01,2022-03-01,1369.0
3393,132,35,"Finance & Insurance, Residential Mortgages",NaN,2021-03-01,2022-03-01,NaN


### Dataset: dropped_members

In [60]:
df_dropped_members = pd.read_csv("data/dropped_members_data.csv", index_col=0, encoding="ISO-8859-1")
column_list = df_dropped_members.columns.tolist()
column_list = column_list[-2:-1] + column_list[:-2]
df_dropped_members = df_dropped_members[column_list]

df_dropped_members.drop(["Date/Time"], axis=1, inplace=True)

df_dropped_members.rename(columns={"Effective Drop Date": "drop_date",
                                   "Reason": "drop_reason",
                                   "Drop Type": "drop_type"}, 
                          inplace=True)
df_dropped_members.head(10)

,user_ID,drop_date,drop_reason,drop_type
0,1936.0,11/01/2021,Takes Too Much Time,Resigned
1,1546.0,24/02/2021,Personal Reasons,Resigned
2,1622.0,05/02/2021,Other Reason (see notes),Resigned
3,2598.0,16/02/2021,Other Reason (see notes),Resigned
4,2760.0,16/02/2021,Member Transferred BNI Chapters,Resigned
5,1178.0,11/02/2021,Other Reason (see notes),Resigned
6,2699.0,09/02/2021,No Reason Entered,Resigned
7,2474.0,01/02/2021,Member Transferred BNI Chapters,Resigned
8,2462.0,31/01/2021,Personal Reasons,Resigned
9,1479.0,01/12/2020,NaN,System


In [61]:
df_dropped_members.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2093 entries, 0 to 2092
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_ID      2077 non-null   float64
 1   drop_date    2093 non-null   object 
 2   drop_reason  1216 non-null   object 
 3   drop_type    2093 non-null   object 
dtypes: float64(1), object(3)
memory usage: 81.8+ KB


Seems like there are some missing values in ```user_ID``` column, which is odd. There shouldn't be any missings values in this table.

In [62]:
df_dropped_members[df_dropped_members["user_ID"].isna()]

,user_ID,drop_date,drop_reason,drop_type
1299,NaN,16/03/2015,NaN,Yes
1395,NaN,25/08/2014,NaN,Yes
1418,NaN,15/07/2014,NaN,Yes
1437,NaN,27/05/2014,NaN,Yes
1440,NaN,21/05/2014,NaN,Yes
1676,NaN,09/10/2012,NaN,Yes
1833,NaN,22/10/2011,NaN,Yes
1834,NaN,22/10/2011,NaN,Yes
1835,NaN,22/10/2011,NaN,Yes
1863,NaN,15/08/2011,NaN,Yes


All those records are listed from a time period before the PALMS data that is being looked into, so those records can be dropped.

In [63]:
df_dropped_members.dropna(subset=["user_ID"], inplace=True)
df_dropped_members.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2077 entries, 0 to 2092
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_ID      2077 non-null   float64
 1   drop_date    2077 non-null   object 
 2   drop_reason  1216 non-null   object 
 3   drop_type    2077 non-null   object 
dtypes: float64(1), object(3)
memory usage: 81.1+ KB


In [64]:
df_dropped_members["drop_date"] = pd.to_datetime(df_dropped_members["drop_date"], format='%d/%m/%Y', errors='coerce')
df_dropped_members["user_ID"] = pd.to_numeric(df_dropped_members["user_ID"], downcast='integer')
df_dropped_members.head()

,user_ID,drop_date,drop_reason,drop_type
0,1936,2021-01-11,Takes Too Much Time,Resigned
1,1546,2021-02-24,Personal Reasons,Resigned
2,1622,2021-02-05,Other Reason (see notes),Resigned
3,2598,2021-02-16,Other Reason (see notes),Resigned
4,2760,2021-02-16,Member Transferred BNI Chapters,Resigned


## Join data - create a master dataframe

In [15]:
df_master = df_palms.copy()
df_master = df_master.merge(df_database.drop("chapter_ID", axis=1), how="left", on="user_ID")
df_master = df_master.merge(df_dropped_members, how="left", on="user_ID")
df_master.drop(["drop_reason", "drop_type", "position", "sponsor_ID"], axis=1, inplace=True)
df_master.sample(10, random_state=13)

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,RRO,V,1-2-1,TYFCB,CEU,palms_date,profession,join_date,renewal_date,drop_date
5068,1226,1,3,0,0,0,0,0,3,0,3,0,2,178,1,2016-10-01,"Financial, Investment Advisor",2016-04-01,2017-10-01,NaT
15797,2547,25,3,0,0,0,2,0,0,1,0,0,1,4838,1,2017-11-01,"Real Estate, Real Estate Sales Representative ...",2014-05-01,2016-06-01,2018-01-18
42547,840,25,1,0,0,0,0,0,0,0,0,0,0,0,0,2020-09-01,"Security, Security Services",2020-09-01,2021-02-01,2019-10-04
16824,1448,9,4,0,0,0,0,2,4,1,2,2,10,0,40,2018-01-01,"Information Technology, Design Web",2017-11-01,2018-11-01,2019-11-01
21369,1311,30,4,0,0,0,0,0,8,0,1,1,10,3952,7,2018-06-01,"Financial, Investment Advisor",2017-04-01,2020-04-01,2020-03-19
43444,2184,5,4,0,0,0,0,5,18,3,6,0,24,25437,4,2020-11-01,"Trades, General Contractor",2018-09-01,2019-09-01,2018-09-12
26955,2911,31,4,0,0,0,1,3,3,1,3,0,8,7329,0,2019-01-01,"Trades, Renovations - Exterior",2017-06-01,2019-06-01,2017-04-01
23692,1004,30,4,0,0,0,0,4,10,1,1,1,6,1518,4,2018-09-01,"Real estate services, Residential Real Estate ...",2015-12-01,2021-12-01,NaT
535,818,24,5,0,0,0,0,3,0,2,6,0,6,80,0,2016-03-01,"Marketing, Digital Marketing",2015-12-01,2018-09-01,2018-05-23
5404,82,16,4,0,0,0,0,3,0,0,1,0,4,1319,11,2016-10-01,"Alternative Medicine, Nutritionist",2013-09-01,2017-09-01,2017-04-17


In [16]:
df_master.shape

(46412, 20)

# Data cleaning & aggregation
## Get relative renewal date for data aggregation

In [17]:
df_master_clean = df_master.copy()
df_master_clean["palms_date"] = pd.to_datetime(df_master_clean["palms_date"], errors='coerce')
df_master_clean["renewal_date"] = pd.to_datetime(df_master_clean["renewal_date"], errors='coerce')

df_master_clean["year_of_membership"] = (df_master_clean["palms_date"] - df_master_clean["join_date"]) / np.timedelta64(1, 'M')
df_master_clean["year_of_membership"] = df_master_clean["year_of_membership"].round().astype(int) // 12
df_master_clean["months_to_renewal"] = (df_master_clean["renewal_date"] - df_master_clean["palms_date"]) / np.timedelta64(1, 'M')
df_master_clean["months_to_renewal"] = df_master_clean["months_to_renewal"].round().astype(int)
df_master_clean["years_to_renewal"] = (df_master_clean["months_to_renewal"] - 1) // 12
df_master_clean["months_to_renewal"] = df_master_clean["months_to_renewal"] % 12

# Substitute "months_to_renewal" value 0 with 12 for ease of aggregation later on
df_master_clean.loc[df_master_clean["months_to_renewal"] == 0, "months_to_renewal"] = 12

for index, row in df_master_clean.iterrows():
    df_master_clean.at[index, 'relative_renewal_date'] = row['renewal_date'] - pd.DateOffset(years=row['years_to_renewal'])

df_master_clean.drop(["years_to_renewal"], axis=1, inplace=True)
df_master_clean.head(10)

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,TYFCB,CEU,palms_date,profession,join_date,renewal_date,drop_date,year_of_membership,months_to_renewal,relative_renewal_date
0,733,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,-1,1,2016-04-01
1,733,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Food/Beverages, Restaurant",2017-05-01,2018-05-01,NaT,-2,2,2016-05-01
2,1150,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Trades, Heating & A/C",2016-04-01,2017-04-01,NaT,-1,1,2016-04-01
3,414,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Insurance, Group Benefits Consultant",2016-04-01,2017-04-01,NaT,-1,1,2016-04-01
4,1721,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Marketing, Marketing Services",2016-04-01,2017-04-01,NaT,-1,1,2016-04-01
5,1919,1,4,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Real Estate, Real Estate Sales Representative ...",2016-04-01,2018-12-01,NaT,-1,9,2016-12-01
6,631,1,3,1,0,0,0,0,0,0,...,0,0,2016-03-01,"Office, Full Service Offices",2016-04-01,2017-10-01,NaT,-1,7,2016-10-01
7,2260,1,2,1,0,0,0,0,0,0,...,0,0,2016-03-01,"Information Technology, Computer Sales and Ser...",2016-04-01,2017-04-01,NaT,-1,1,2016-04-01
8,313,1,2,1,0,0,0,0,0,0,...,0,0,2016-03-01,"Health & Wellness, Gym",2016-04-01,2018-09-01,NaT,-1,6,2016-09-01
9,1267,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Accounting, Bookkeeper",2016-04-01,2019-04-01,NaT,-1,1,2016-04-01


## Checks records
 1. if ```year_of_membership``` is a negative number - **wrong**
 2. if ```palms_date``` is greater than ```join_date``` - **wrong**

Perhaps solving item **1** will also solve **2**. Let's tackle the first check:
 
 
#### 1. Check if ```year_of_membership``` is a negative number
A bit of context: BNI chapters can be split into two categories:
1. fully established
2. forming (known as "core groups")

BNI's membership model for core groups is that an approved applicant pays for one-year membership when he first joins the group, but the ```join_date``` is set to the date of the group "launch" - when it becomes fully established as a chapter. That being said the PALMS data can be entered for core group but it only becomes available after the group is launched. For this reason there is no PALMS data for ```chapter_ID == 2```, which is a core group that has not been launched yet:

In [18]:
df_master_clean["chapter_ID"].max()

31

In [19]:
df_master_clean.loc[df_master["chapter_ID"] == 2]

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,TYFCB,CEU,palms_date,profession,join_date,renewal_date,drop_date,year_of_membership,months_to_renewal,relative_renewal_date


Unfortunately the available data does not contain information about launch date of each group nor the actual date of the actual core group member join date.

**Further discussion:** the core group members join date problem extends even further because it creates a false notion that a membership is shorter than it actually has been. An example of this can be found  in ```chapter_ID == 27``` which is a core group that has been launched sometime between 2019 and 2020:

In [20]:
df_master_clean.loc[df_master_clean["chapter_ID"] == 27].drop_duplicates("user_ID")["join_date"].value_counts().sort_index()

2016-02-01     1
2019-02-01     1
2019-10-01     1
2020-03-01    15
2020-06-01     2
2020-07-01     2
2020-10-01     1
2020-12-01     1
2021-02-01     4
Name: join_date, dtype: int64

The majority of the group, 15 members have a ```join_date``` set to **March 1st, 2020** which is most likely the date when the chapter launched. This means that any ```join_date``` after March 1st is not problematic as the person joined the group after it was launched and it is the true date of when the member joined.

There is, however, a significant problem for members who started their membership before and during March 1st. It is unclear why there are some members with a ```join_date``` prior to March 1st. Regardless, in both cases of members' ```join_date``` set to March 1st or prior to that date one cannot be sure if it is the true date when the member actually joined.

This is not the case with a well established chapter such as ```chapter_ID == 8```:

In [21]:
df_master_clean.loc[df_master_clean["chapter_ID"] == 8].drop_duplicates("user_ID")["join_date"].value_counts().sort_index()

1998-05-01    1
2002-09-01    1
2006-11-01    1
2010-08-01    1
2011-09-01    1
             ..
2020-05-01    1
2020-07-01    1
2020-12-01    2
2021-01-01    2
2021-02-01    1
Name: join_date, Length: 61, dtype: int64

Lets have a look which chapters contain members with a negative ```year_of_membership```.

In [22]:
print(sorted(df_master_clean.loc[df_master_clean["year_of_membership"] < 0, "chapter_ID"].unique()))

[1, 3, 5, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 20, 22, 23, 24, 25, 27, 28, 30, 31]


I thought that it was only core groups but it seems that fully established chapters have members with negative ```year_of_membership``` as well. Let's look into members who have any records with negative ```year_of_membership``` and try to fix them by cross-checking with original data.

In [65]:
df_master_clean.loc[df_master_clean["year_of_membership"] < 0]

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,TYFCB,CEU,palms_date,profession,join_date,renewal_date,drop_date,year_of_membership,months_to_renewal,relative_renewal_date
0,733,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,-1,1,2016-04-01
1,733,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Food/Beverages, Restaurant",2017-05-01,2018-05-01,NaT,-2,2,2016-05-01
2,1150,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Trades, Heating & A/C",2016-04-01,2017-04-01,NaT,-1,1,2016-04-01
3,414,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Insurance, Group Benefits Consultant",2016-04-01,2017-04-01,NaT,-1,1,2016-04-01
4,1721,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Marketing, Marketing Services",2016-04-01,2017-04-01,NaT,-1,1,2016-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45967,1396,14,1,0,0,0,0,0,2,0,...,0,0,2021-02-01,"Real estate services, Home Inspection",2021-03-01,2022-03-01,NaT,-1,1,2021-03-01
46001,1799,15,2,0,0,0,0,0,0,0,...,0,0,2021-02-01,"Finance & Insurance, Supplemental Insurance",2021-03-01,2022-03-01,NaT,-1,1,2021-03-01
46005,17,15,1,0,0,0,0,0,0,0,...,0,0,2021-02-01,"Health & Wellness, Chiropractor",2021-03-01,2022-03-01,NaT,-1,1,2021-03-01
46010,2195,16,1,0,0,0,0,0,0,0,...,0,0,2021-02-01,"Advertising & Marketing, Photographer",2021-03-01,2022-03-01,NaT,-1,1,2021-03-01


In [66]:
df_master_clean.loc[df_master_clean["year_of_membership"] < 0, "user_ID"].unique()

array([ 733, 1150,  414, 1721, 1919,  631, 2260,  313, 1267, 1870,  920,
       1320,  740, 1074, 2105, 1226, 2162, 1764, 2471,  540,  479, 2612,
       2764,  979, 1229, 1509,  282, 2760, 2529, 1964, 2475, 1895, 2127,
        895, 1186, 1997, 1925, 2029, 1395, 1476,  676, 1217,  136,  577,
        839, 1345,  516, 2237, 2883, 2279, 1141, 2464,   60, 1359, 1875,
        475, 2906, 2053, 2613, 2164, 1384, 1314, 1465,   99, 2376,  454,
       1960, 1898,  818, 1381,  581,  250,  840,  645, 1907, 2051, 2276,
       2911, 2846, 2370, 1643, 1136, 1223,  790,  108,  816, 2679, 1646,
       1168, 1953, 1957, 2648, 2568, 1519,  768, 2838,  970, 2721,  651,
       2358, 1081, 1712, 2681,  432,  219, 1146, 2709,  443, 2205, 1832,
        258, 2015, 1848, 2791, 2184,   45, 1033,  902,  344, 2546, 1671,
       1236,    9, 1659,  668, 1421,  554,  658, 2640,  774, 2513,  176,
       2024, 2497, 1268, 2549,   18, 1485,  941, 1714, 2292, 2394, 1916,
       1201, 2891, 2040,  844,  104,  752,  327, 18

There seems to be many records that have a negative ```year_of_membership``` and it would be too tedious too look into every single one specifically. There seems to be a pattern however that the first month in PALMS when the member joins is prior to the actual ```join_date```. Having this in mind, I recall that a member was allowed to start attending chapter meetings a little bit before he was entered into the system. Nonetheless, records with one or two months prior to the members' ```join_date``` aren't a problem as they can safely be dropped and excluded from the aggregation later on.

There is, however, a **problem if a member has more that two months of negative** ```year_of_membership``` because it might indicate a different situation than above. Let's create a new dataframe which counts how many PALMS records contain a negative ```year_of_membership``` per user.

In [68]:
df_negative = df_master_clean.copy()
df_negative["negative_months"] = 1
df_negative = df_negative.loc[df_negative["year_of_membership"] < 0].groupby("user_ID")[["negative_months"]].count()
df_negative

,negative_months
user_ID,
6,210
9,1
17,1
18,4
26,1
...,...
2906,53
2911,32
2915,1


There are 450 users with at least one month with negative ```year_of_membership```. First lets look into those who have 2 or less records with ```negative_months``` to ensure that they are indeed just a few months before the actual join date.

## TODO

Now lets look into members who have **at least two** ```negative_months```

In [69]:
df_negative.loc[df_negative["negative_months"] > 2].sort_values("negative_months", ascending=False)

,negative_months
user_ID,
2184,276
136,261
6,210
1895,168
1186,147
...,...
2324,4
353,4
2431,4


That leaves us with only 121 users to look into. Lets try to merge those users with the ```df_database``` to obtain the ```chapter_ID``` for all users. This will allow for determining if there are any group-related patterns.

In [70]:
df_database.loc[df_database["user_ID"] == 2053, ["user_ID", "chapter_ID"]]

,user_ID,chapter_ID
2032,2053,16
2439,2053,18
2440,2053,18


In [71]:
df_negative = df_negative.loc[df_negative["negative_months"] > 2].sort_values("negative_months", ascending=False)

df_negative = df_negative.merge(df_database[["user_ID", "chapter_ID"]],
                                how="left",
                                left_index=True,
                                right_on="user_ID")

df_negative

,negative_months,user_ID,chapter_ID
1801,276,2184,5
1804,276,2184,5
1841,276,2184,5
1842,276,2184,5
1880,276,2184,5
...,...,...,...
1149,4,2662,25
1966,4,2662,16
2032,3,2053,16
2439,3,2053,18


Unexpectedly the number of rows of the dataframe has grown from 122 to 294, which means that there are som duplicate entries in the ```df_database```. This is most likely due to some account merges or incorrect entries. Lets filter out the duplicate entries and pay close attention to the original data to find and correct any inconsistencies.

In [72]:
df_negative.drop_duplicates("user_ID", inplace=True)
df_negative.set_index("user_ID", inplace=True)
df_negative

,negative_months,chapter_ID
user_ID,,
2184,276,5
136,261,10
6,210,25
1895,168,8
1186,147,9
...,...,...
2324,4,5
353,4,18
2431,4,3


The number of observations has gone back to 122. Let's go back and try to find any chapter-related pattern. Are there any chapters which have more ```negative_months``` than others?

In [73]:
df_negative.groupby("chapter_ID").count()

,negative_months
chapter_ID,
1,3
3,4
5,13
8,7
9,10
10,6
11,9
14,2
16,6


In [74]:
df_master_clean["chapter_ID"].max()

31

In [77]:
df_negative[df_negative["chapter_ID"] == 39]

,negative_months,chapter_ID
user_ID,,
2872,39,39


In [78]:
df_master_clean[df_master_clean["user_ID"] == 2872]

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,TYFCB,CEU,palms_date,profession,join_date,renewal_date,drop_date,year_of_membership,months_to_renewal,relative_renewal_date
16963,2872,13,2,0,0,0,0,1,1,0,...,0,0,2018-01-01,"Mortgage, Mortgage Agent",2016-12-01,2017-12-01,2019-07-24,1,11,2018-12-01
16964,2872,13,2,0,0,0,0,1,1,0,...,0,0,2018-01-01,"Mortgage, Residential Mortgage",2018-02-01,2019-10-01,2019-07-24,-1,9,2018-10-01
16965,2872,13,2,0,0,0,0,1,1,0,...,0,0,2018-01-01,"Mortgage, Residential Mortgage",2020-07-01,2020-09-01,2019-07-24,-3,8,2018-09-01
16966,2872,13,2,0,0,0,0,1,1,0,...,0,0,2018-01-01,"Mortgage, Broker",2020-07-01,2021-07-01,2019-07-24,-3,6,2018-07-01
17780,2872,13,3,0,0,0,1,4,12,0,...,44558,8,2018-02-01,"Mortgage, Mortgage Agent",2016-12-01,2017-12-01,2019-07-24,1,10,2018-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30688,2872,13,4,0,0,0,0,0,4,0,...,0,1,2019-06-01,"Mortgage, Broker",2020-07-01,2021-07-01,2019-07-24,-2,1,2019-07-01
31522,2872,13,5,0,0,0,0,0,2,0,...,787,10,2019-07-01,"Mortgage, Mortgage Agent",2016-12-01,2017-12-01,2019-07-24,2,5,2019-12-01
31523,2872,13,5,0,0,0,0,0,2,0,...,787,10,2019-07-01,"Mortgage, Residential Mortgage",2018-02-01,2019-10-01,2019-07-24,1,3,2019-10-01
31524,2872,13,5,0,0,0,0,0,2,0,...,787,10,2019-07-01,"Mortgage, Residential Mortgage",2020-07-01,2020-09-01,2019-07-24,-1,2,2019-09-01


### Cleaning single users:
#### - User 2184

In [156]:
df_master_clean.loc[df_master_clean["user_ID"] == 2184].head(10)

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,TYFCB,CEU,palms_date,profession,join_date,renewal_date,drop_date,year_of_membership,months_to_renewal,relative_renewal_date
3651,2184,5,2,0,0,0,0,0,10,1,...,3319,3,2016-08-01,"Alternative Medicine, Osteopath",2016-08-01,2017-08-01,2019-08-27,0,12,2017-08-01
3652,2184,5,2,0,0,0,0,0,10,1,...,3319,3,2016-08-01,"Alternative Medicine, Osteopath",2016-08-01,2017-08-01,2018-09-12,0,12,2017-08-01
3653,2184,5,2,0,0,0,0,0,10,1,...,3319,3,2016-08-01,"Alternative Medicine, Osteopath",2016-08-01,2017-08-01,2018-08-01,0,12,2017-08-01
3654,2184,5,2,0,0,0,0,0,10,1,...,3319,3,2016-08-01,"Health & Wellness, Massage Therapist",2016-08-01,2018-08-01,2019-08-27,0,12,2017-08-01
3655,2184,5,2,0,0,0,0,0,10,1,...,3319,3,2016-08-01,"Health & Wellness, Massage Therapist",2016-08-01,2018-08-01,2018-09-12,0,12,2017-08-01
3656,2184,5,2,0,0,0,0,0,10,1,...,3319,3,2016-08-01,"Health & Wellness, Massage Therapist",2016-08-01,2018-08-01,2018-08-01,0,12,2017-08-01
3657,2184,5,2,0,0,0,0,0,10,1,...,3319,3,2016-08-01,"Trades, General Contractor",2018-09-01,2019-09-01,2019-08-27,-3,1,2016-09-01
3658,2184,5,2,0,0,0,0,0,10,1,...,3319,3,2016-08-01,"Trades, General Contractor",2018-09-01,2019-09-01,2018-09-12,-3,1,2016-09-01
3659,2184,5,2,0,0,0,0,0,10,1,...,3319,3,2016-08-01,"Trades, General Contractor",2018-09-01,2019-09-01,2018-08-01,-3,1,2016-09-01
3660,2184,5,2,0,0,0,0,0,10,1,...,3319,3,2016-08-01,"Trades, General Contractor",2018-09-01,2019-09-01,2019-08-27,-3,1,2016-09-01


This user has the same ```chapter_ID``` for all of his records but a varying ```drop_date```. Having a look at the original data, this persons records have been merged and there 

#### - User 733

In [133]:
df_master_clean.loc[df_master_clean["user_ID"] == 733].head(10).sort("year_of_membership")

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,TYFCB,CEU,palms_date,profession,join_date,renewal_date,drop_date,year_of_membership,months_to_renewal,relative_renewal_date
0,733,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,-1,1,2016-04-01
1,733,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Food/Beverages, Restaurant",2017-05-01,2018-05-01,NaT,-2,2,2016-05-01
717,733,1,3,1,0,0,0,0,0,4,...,0,16,2016-04-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,12,2017-04-01
718,733,1,3,1,0,0,0,0,0,4,...,0,16,2016-04-01,"Food/Beverages, Restaurant",2017-05-01,2018-05-01,NaT,-2,1,2016-05-01
1426,733,1,3,0,0,0,1,0,0,2,...,0,0,2016-05-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,11,2017-04-01
1427,733,1,3,0,0,0,1,0,0,2,...,0,0,2016-05-01,"Food/Beverages, Restaurant",2017-05-01,2018-05-01,NaT,-1,12,2017-05-01
2144,733,1,3,2,0,0,0,0,0,2,...,294,0,2016-06-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,10,2017-04-01
2145,733,1,3,2,0,0,0,0,0,2,...,294,0,2016-06-01,"Food/Beverages, Restaurant",2017-05-01,2018-05-01,NaT,-1,11,2017-05-01
2874,733,1,3,0,0,0,0,0,0,1,...,0,0,2016-07-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,9,2017-04-01
2875,733,1,3,0,0,0,0,0,0,1,...,0,0,2016-07-01,"Food/Beverages, Restaurant",2017-05-01,2018-05-01,NaT,-1,10,2017-05-01


```user_ID = 733``` - this member a couple of records. The one with ```renewal_date``` set to ```2018-05-01``` is invalid.

In [134]:
df_master_clean = df_master_clean.loc[~((df_master_clean["user_ID"] == 733) & (df_master_clean["join_date"] == "2017-05-01"))]
df_master_clean.loc[df_master_clean["user_ID"] == 733].head(10)

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,TYFCB,CEU,palms_date,profession,join_date,renewal_date,drop_date,year_of_membership,months_to_renewal,relative_renewal_date
0,733,1,3,0,0,0,0,0,0,0,...,0,0,2016-03-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,-1,1,2016-04-01
717,733,1,3,1,0,0,0,0,0,4,...,0,16,2016-04-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,12,2017-04-01
1426,733,1,3,0,0,0,1,0,0,2,...,0,0,2016-05-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,11,2017-04-01
2144,733,1,3,2,0,0,0,0,0,2,...,294,0,2016-06-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,10,2017-04-01
2874,733,1,3,0,0,0,0,0,0,1,...,0,0,2016-07-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,9,2017-04-01
3582,733,1,3,0,1,0,0,0,0,1,...,0,0,2016-08-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,8,2017-04-01
4315,733,1,5,0,0,0,0,1,1,6,...,0,2,2016-09-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,7,2017-04-01
5048,733,1,3,0,0,0,0,0,1,1,...,0,2,2016-10-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,6,2017-04-01
5809,733,1,4,0,0,0,0,0,1,0,...,0,14,2016-11-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,5,2017-04-01
6555,733,1,5,0,0,0,0,2,1,7,...,0,27,2016-12-01,"Food/Beverages, Caterer",2016-04-01,2017-04-01,NaT,0,4,2017-04-01


Ok, this is good. There is still one month with a negative ```year_of_membership``` but it won't be taken into account in the calculations anyway so it can be dropped.

## Aggregate 3-months data
Ensure sure that each groupby sum is aggregated for 3 months, not 2 or 1

In [38]:
df_3_months = df_master.copy()

df_3_months = df_3_months.loc[(df_3_months["months_to_renewal"] <= 6) & (df_3_months["months_to_renewal"] >= 4)]

# df_3_months.drop(["years_to_renewal", "months_to_renewal"], axis=1, inplace=True)

df_3_months.loc[df_3_months["chapter_ID"] == 27, :]

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,CEU,palms_date,profession,join_date,renewal_date,drop_date,year_of_membership,months_to_renewal,years_to_renewal,next_renewal_date
37248,734,27,1,0,0,0,0,0,0,0,...,0,2020-02-01,"Health & Wellness, Massage Therapist",2020-03-01,2021-08-01,NaT,-1,6,1,2020-08-01
38028,734,27,4,0,0,0,0,2,3,0,...,11,2020-03-01,"Health & Wellness, Massage Therapist",2020-03-01,2021-08-01,NaT,0,5,1,2020-08-01
38803,822,27,5,0,0,0,0,3,0,2,...,0,2020-04-01,"Office, Commercial Painting",2019-10-01,2020-10-01,NaT,0,6,0,2020-10-01
38809,2904,27,5,0,0,0,0,0,2,0,...,7,2020-04-01,"Legal & Accounting, Bookkeeping",2020-03-01,2021-10-01,NaT,0,6,1,2020-10-01
38816,734,27,5,0,0,0,0,3,3,2,...,13,2020-04-01,"Health & Wellness, Massage Therapist",2020-03-01,2021-08-01,NaT,0,4,1,2020-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46298,734,27,4,0,0,0,0,0,4,5,...,8,2021-02-01,"Health & Wellness, Massage Therapist",2020-03-01,2021-08-01,NaT,0,6,0,2021-08-01
46299,1580,27,4,0,0,0,0,1,4,1,...,3,2021-02-01,"Advertising & Marketing, Photographer",2020-07-01,2021-07-01,2020-03-18,0,5,0,2021-07-01
46300,1580,27,4,0,0,0,0,1,4,1,...,3,2021-02-01,"Advertising & Marketing, Photographer",2020-07-01,2021-07-01,2020-09-28,0,5,0,2021-07-01
46301,1580,27,4,0,0,0,0,1,4,1,...,3,2021-02-01,"Advertising & Marketing, Photographer",2020-07-01,2021-07-01,2020-03-18,0,5,0,2021-07-01


In [21]:
# group by "renewal_date"
# The rest of the columns listed in groupby remain the same for the given time period.
test = df_3_months.copy()
test = test.groupby(["user_ID", "renewal_date"]).sum()
# test.reset_index(level=["renewal_date"], inplace=True)
test.head()

chapter_ID   P  A  L  M  S  RGI  RGO  RRI  RRO  V  \
user_ID renewal_date                                                      
2       2017-05-01            51  11  0  0  0  0    1   13    5   12  0   
        2018-05-01            51  12  0  0  0  0    2    3    2    6  0   
        2019-05-01            51  10  0  0  0  0    2    3    3    4  1   
        2020-05-01            51  10  0  0  0  1    4    3    6   27  3   
        2021-05-01            51   9  1  0  0  0    0    9    5    4  0   

                      1-2-1  TYFCB  CEU  year_of_membership  
user_ID renewal_date                                         
2       2017-05-01       10   4503    1                   3  
        2018-05-01       12   4027    4                   6  
        2019-05-01        5   1445    2                   9  
        2020-05-01        9   1873    9                  12  
        2021-05-01        4   1000    1                  15

In [125]:
test.loc[test["user_ID"] == 420]

,user_ID,chapter_ID,profession,membership_length,P,A,L,M,S,RGI,RGO,RRI,RRO,V,1-2-1,TYFCB,CEU
renewal_date,,,,,,,,,,,,,,,,,
2016-07-01,420,8,"Finance & Insurance, Financial Advisor/Financi...",17,4,1,0,0,0,1,1,0,0,0,1,3195,1
2017-07-01,420,8,"Finance & Insurance, Financial Advisor/Financi...",18,9,1,0,0,3,1,2,2,1,0,16,7421,3
2018-07-01,420,8,"Finance & Insurance, Financial Advisor/Financi...",19,11,1,0,0,0,9,8,1,8,1,15,21651,23
2019-07-01,420,8,"Finance & Insurance, Financial Advisor/Financi...",20,9,1,0,0,2,1,8,1,18,1,7,3734,11
2020-07-01,420,8,"Finance & Insurance, Financial Advisor/Financi...",21,11,0,0,0,0,2,14,0,8,1,7,1796,14
2021-07-01,420,8,"Finance & Insurance, Financial Advisor/Financi...",22,7,0,0,0,0,2,11,2,7,1,9,3891,11
